In [2]:
import sys,re
import numpy as np
from sklearn import linear_model
from scipy import sparse
from collections import Counter
from sklearn import grid_search
from sklearn import cross_validation


In [69]:
L2_REGULARIZATION_STRENGTH = 1e0
MINIMUM_FEAT_COUNT=5

# dictionary of feature: index
# feature = a dictionary of feature name: value
feature_vocab={}

# reverse_features[index] = feature
reverse_features=[]

# dictionary of label: index, where
# label = SHIFT, LEFTARC_DEPENDENCY_LABEL, RIGHTARC_DEPENDENCY_LABEL
label_vocab={}

# reverse_labels[index] = label
reverse_labels=[]

# number/ID of features
fmax = 0

# 1. Checking for Projectivity 

In [4]:
def is_projective(toks):
    """
    params: toks is a list of (idd, tok, pos, head, lab) for a sentence
    return True if and only if the sentence has a projective dependency tree
    """

    # Implement your code below
    deps = {}
    heads = {}
    for (idd, _, _, head, _) in toks:
        if head not in deps:
            deps[head] = []
        deps[head].append(idd)
        heads[idd] = head

    for key,value in deps.items():
        for dep in value:
            inv = [dep,key]
            for des in range(min(inv)+1,max(inv)):
                if heads[des] not in range(min(inv),max(inv)+1):
                    return False
                
    return True
    

In [5]:
#Test Case 1 For Checking for Projectivity
toks1 = [(1, 'I', _, 2, _),\
        (2, 'love', _, 0, _),\
        (3, 'Erica', _, 2, _),\
        (4, 'who', _, 3, _),\
        (5, 'is', _, 4, _),\
        (6, 'very', _, 7, _),\
        (7, 'pretty', _, 4, _)]
print(is_projective(toks1))

toks2 = [(1, 'JetBlue', _, 2, _),\
        (2, 'canceled', _, 0, _),\
        (3, 'our', _, 4, _),\
        (4, 'flight', _, 2, _),\
        (5, 'this', _, 6, _),\
        (6, 'morning', _, 2, _),\
        (7, 'which', _, 8, _), \
        (8, 'was', _, 4, _), \
        (9, 'already', _, 10, _), \
        (10, 'late', _, 8, _)]
print(is_projective(toks2))

toks3 = [(1, 'JetBlue', _, 2, _),\
        (2, 'canceled', _, 0, _),\
        (3, 'our', _, 4, _),\
        (4, 'flight', _, 2, _),\
        (5, 'this', _, 2, _),\
        (6, 'morning', _, 8, _),\
        (7, 'which', _, 8, _), \
        (8, 'was', _, 11, _), \
        (9, 'already', _, 11, _), \
        (10, 'already', _, 11, _), \
        (11, 'already', _, 2, _), \
        (12, 'already', _, 11, _), \
        (13, 'late', _, 2, _)]
print(is_projective(toks3))


True
False
True


In [6]:
#Test Case 2 For Checking for Projectivity
with open("test.conll") as f:
    toks = []
    
    for line in f:
        cols = line.rstrip().split("\t")

        if len(cols) < 2: # at the end of each sentence
            if len(toks) > 0:
                print(is_projective(toks))
                
                toks, tokens, postags = [], {}, {}
                tokens[0] = "ROOT"
                postags[0] = "ROOT"
                continue

            if cols[0].startswith("#"):
                continue

        idd, tok, pos, head, lab = int(cols[0]), cols[1], cols[4], int(cols[6]), cols[7]
        toks.append((idd, tok, pos, head, lab))
        

True
True


# 2. Creating an Oracle

In [38]:

# Question 2.a.
def perform_shift(wbuffer, stack, arcs,
                  configurations, gold_transitions):
    """
    perform the SHIFT operation
    """

    # Implement your code below
    # your code should:
    # 1. append the latest configuration to configurations
    # 2. append the latest action to gold_transitions
    # 3. update wbuffer, stack and arcs accordingly
    # hint: note that the order of operations matters
    # as we want to capture the configurations and transition rules
    # before making changes to the stack, wbuffer and arcs
    
    configurations.append((wbuffer.copy(), stack.copy(), arcs.copy()))
    gold_transitions.append("SHIFT")
    stack.append(wbuffer[-1])
    wbuffer.pop(-1)


# Question 2.b.
def perform_arc(direction, dep_label,
                wbuffer, stack, arcs,
                configurations, gold_transitions):
    """
    params:
        - direction: {"LEFT", "RIGHT"}
        - dep_label: label for the dependency relations
    Perform LEFTARC_ and RIGHTARC_ operations
    """

    # Implement your code below
    # your code should:
    # 1. append the latest configuration to configurations
    # 2. append the latest action to gold_transitions
    # 3. update wbuffer, stack and arcs accordingly
    # hint: note that the order of operations matters
    # as we want to capture the configurations and transition rules
    # before making changes to the stack, wbuffer and arcs
    
    configurations.append((wbuffer.copy(), stack.copy(), arcs.copy()))
    string = direction+"ARC_"+str(dep_label)
    gold_transitions.append(string)
    
    if direction == "LEFT":
        arcs.append((dep_label, stack[-1], stack[-2]))
        stack.pop(-2)
    elif direction == "RIGHT":
        arcs.append((dep_label, stack[-2], stack[-1]))
        stack.pop(-1)
        


# Question 2
def tree_to_actions(wbuffer, stack, arcs, deps):
    """
    params:
    wbuffer: a list of word indices, top of buffer is at the end of the list
    stack: a list of word indices, top of buffer is at the end of the list
    arcs: a list of (label, head, dependent) tuples
    deps: {head1:{
                   (head1, child1):dependency_label1,
                   (head1, child2):dependency_label2
                 }
           head2:{
                   (head2, child3):dependency_label3,
                   (head2, child4):dependency_label4
                 }
          }

    Given wbuffer, stack, arcs and deps
    Return configurations and gold_transitions (actions)
    """

    # configurations:
    # A list of tuples of lists
    # [(wbuffer1, stack1, arcs1), (wbuffer2, stack2, arcs2), ...]
    # Keeps tracks of the states at each step
    configurations=[]

    # gold_transitions:
    # A list of action strings, e.g ["SHIFT", "LEFTARC_nsubj"]
    # Keeps tracks of the actions at each step
    gold_transitions=[]

    # Implement your code below
    # hint:
    # 1. configurations[i] and gold_transitions[i] should
    # correspond to the states of the wbuffer, stack, arcs
    # (before the action was take) and action to take at step i
    # 2. you should call perform_shift and perform_arc in your code
    
    #To put first word in stack
    perform_shift(wbuffer, stack, arcs, configurations, gold_transitions)

    while stack != [0]:
        
        ind = False
        #check action leftarc
        if stack[-1] in deps:
            for key,value in deps[stack[-1]].items():
                dep_label = value
                if key[1] == stack[-2]:
                    ind = True
                    break
            if ind:
                perform_arc("LEFT", dep_label, wbuffer, stack, arcs, configurations, gold_transitions)
                continue
                    
        #check action rightarc
        if stack[-2] in deps:
            for key,value in deps[stack[-2]].items():
                dep_label = value
                if key[1] == stack[-1]:
                    if stack[-1] in deps:
                        current_deps = [arc[2] for arc in arcs if arc[1] == stack[-1]]
                        all_deps = [key[1] for key in deps[stack[-1]].keys()]
                        if set(current_deps) == set(all_deps):
                            ind = True
                            break
                    else:
                        ind = True
                        break
            if ind:
                perform_arc("RIGHT", dep_label, wbuffer, stack, arcs, configurations, gold_transitions)
                continue
        
        #else action shift
        perform_shift(wbuffer, stack, arcs, configurations, gold_transitions)
    
    return configurations, gold_transitions
        
    

In [39]:
feats, labels = get_oracles("test.conll")

# 3. Tree Parsing with Predictions

In [75]:
def isvalid(stack, wbuffer, action):
    """
    Helper function that returns True only if an action is
    legal given the current states of the stack and wbuffer
    """
    if action == "SHIFT" and len(wbuffer) > 0:
        return True
    if action.startswith("RIGHTARC") and len(stack) > 1 and stack[-1] != 0:
        return True
    if action.startswith("LEFTARC") and len(stack) > 1 and stack[-2] != 0:
        return True

    return False


# Question 3
def action_to_tree(tree, predictions, wbuffer, stack, arcs):
    """
    params:
    tree:
    a dictionary of dependency relations (head, dep_label)
        {
            child1: (head1, dep_lebel1), 
            child2: (head2, dep_label2), ...
        }

    predictions:
    a numpy column vector of probabilities for different dependency labels
    as ordered by the global variable reverse_labels
    predictions.shape = (1, total number of dependency labels)

    wbuffer: a list of word indices, top of buffer is at the end of the list
    stack: a list of word indices, top of buffer is at the end of the list
    arcs: a list of (label, head, dependent) tuples

    """
    global reverse_labels
    
    # Implement your code below
    # hint:
    # 1. the predictions contains the probability distribution for all
    # possible actions for a single step, and you should choose one
    # and update the tree only once
    # 2. some actions predicted are not going to be valid
    # (e.g., shifting if nothing is on the buffer)
    # so sort probs and keep going until we find one that is valid.
    
    #Sort probs with ascending order
    ranks = np.argsort(predictions).flatten().tolist()
    
    while not len(ranks) == 0:
        
        #Choose the rank 1 action
        action = reverse_labels[ranks[-1]]
        if isvalid(stack, wbuffer, action):
            idx = action.find('_')
            if idx is not -1:
                dep_label = action[idx+1:]
                #Action is RIGHT
                if action.startswith("RIGHTARC"):
                    tree[stack[-1]] = (stack[-2], dep_label)
                    arcs.append((dep_label, stack[-2], stack[-1]))
                    stack.pop(-1)
                #Action is LEFT
                if action.startswith("LEFTARC"):
                    tree[stack[-2]] = (stack[-1], dep_label)
                    arcs.append((dep_label, stack[-1], stack[-2]))
                    stack.pop(-2)
            else:
                #Action is SHIFT
                stack.append(wbuffer[-1])
                wbuffer.pop(-1)
            break

        else:
            #If not valid, pop the option
            ranks.pop(-1)
    

# 4. Feature Configurations

In [13]:
# Bonus Question
def featurize_configuration(configuration, tokens, postags):
    """
    !EXTRA CREDIT!:
    Add new features here to improve the performance of the parser

    Given configurations of the stack, input buffer and arcs,
    words of the sentence and POS tags of the words,
    return some features
    """
    wbuffer, stack, arcs = configuration
    feats = {}

    feats["%s_%s" % ("len_buffer", len(wbuffer))] = 1
    feats["%s_%s" % ("len_stack", len(stack))] = 1

    # single-word features
    if len(stack) > 0:
        feats["%s_%s" % ("stack_word_1", tokens[stack[-1]])] = 1
        feats["%s_%s" % ("stack_tag_1", postags[stack[-1]])] = 1
        feats["%s_%s_%s" % ("stack_tag_word_1", tokens[stack[-1]], postags[stack[-1]])] = 1

    if len(stack) > 1:
        feats["%s_%s" % ("stack_word_2", tokens[stack[-2]])] = 1
        feats["%s_%s" % ("stack_tag_2", postags[stack[-2]])] = 1
        feats["%s_%s_%s" % ("stack_tag_word_2", tokens[stack[-2]], postags[stack[-2]])] = 1

    if len(wbuffer) > 0:
        feats["%s_%s" % ("buffer_word_1", tokens[wbuffer[-1]])] = 1
        feats["%s_%s" % ("buffer_tag_1", postags[wbuffer[-1]])] = 1
        feats["%s_%s_%s" % ("buffer_tag_word_1", tokens[wbuffer[-1]], postags[wbuffer[-1]])] = 1

    # word-pair features
    if len(stack) > 1:
        feats["%s_%s_%s_%s" % ("stack1_word_tag_stack2_tag", tokens[stack[-1]], postags[stack[-1]], postags[stack[-2]])] = 1

    return feats


============================================================
# THE FOLLOWING CODE IS PROVIDED
============================================================

## 1. get_oracle(toks)
### return configurations, gold_transitions

In [9]:
def get_oracle(toks):
    """
    Return pairs of configurations + gold transitions (actions)
    from training data
    configuration = a list of tuple of:
        - buffer (top of buffer is at the end of the list)
        - stack (top of buffer is at the end of the list)
        - arcs (a list of (label, head, dependent) tuples)
    gold transitions = a list of actions, e.g. SHIFT
    """

    stack = [] # stack
    arcs = [] # existing list of arcs
    wbuffer = [] # input buffer

    # deps is a dictionary of head: dependency relations, where
    # dependency relations is a dictionary of the (head, child): label
    # deps = {head1:{
    #               (head1, child1):dependency_label1,
    #               (head1, child2):dependency_label2
    #              }
    #         head2:{
    #               (head2, child3):dependency_label3,
    #               (head2, child4):dependency_label4
    #              }
    #         }
    deps = {}

    # ROOT
    stack.append(0)

    # initialize variables
    for position in reversed(toks):
        (idd, _, _, head, lab) = position

        dep = (head, idd)
        if head not in deps:
            deps[head] = {}
        deps[head][dep] = lab

        wbuffer.append(idd)

    # configurations:
    # A list of (wbuffer, stack, arcs)
    # Keeps tracks of the states at each step
    # gold_transitions:
    # A list of action strings ["SHIFT", "LEFTARC_nsubj"]
    # Keeps tracks of the actions at each step
    configurations, gold_transitions = tree_to_actions(wbuffer, stack, arcs, deps)
    return configurations, gold_transitions

## 2. get_oracles(filename)
### return feats, labels

In [10]:
def get_oracles(filename):
    """
    Get configurations, gold_transitions from all sentences
    """
    with open(filename) as f:
        toks, tokens, postags = [], {}, {}
        tokens[0] = "ROOT"
        postags[0] = "ROOT"

        # a list of all features for each transition step
        feats = []
        # a list of labels, e.g. SHIFT, LEFTARC_DEP_LABEL, RIGHTARC_DEP_LABEL
        labels = []

        for line in f:
            cols = line.rstrip().split("\t")

            if len(cols) < 2: # at the end of each sentence
                if len(toks) > 0:
                    if is_projective(toks): # only use projective trees
                        # get all configurations and gold standard transitions
                        configurations, gold_transitions = get_oracle(toks)

                        for i in range(len(configurations)):
                            feat = featurize_configuration(configurations[i], tokens, postags)
                            label = gold_transitions[i]
                            feats.append(feat)
                            labels.append(label)

                    # reset vars for the next sentence
                    toks, tokens, postags = [], {}, {}
                    tokens[0] = "ROOT"
                    postags[0] = "ROOT"
                continue

            if cols[0].startswith("#"):
                continue

            # construct the tuple for each word in the sentence
            # for each word in the sentence
            # idd: index of a word in a sentence, starting from 1
            # tok: the word itself
            # pos: pos tag for that word
            # head: parent of the dependency
            # lab: dependency relation label
            idd, tok, pos, head, lab = int(cols[0]), cols[1], cols[4], int(cols[6]), cols[7]
            toks.append((idd, tok, pos, head, lab))

            # feature for training to predict the gold transition
            tokens[idd], postags[idd] = tok, pos

        return feats, labels

## 3. train, parse, evaluate

In [54]:
def train(feats, labels):
    """
    Train transition-based parsed to predict next action (labels)
    given current configuration (featurized by feats)
    Return the classifier trained using the logistic regression model
    """
    global feature_vocab, label_vocab, fmax, reverse_labels, reverse_features

    lid = 0 # label ID
    D = len(feats) # each row of feats corresponds to a row in labels
    feature_counts = Counter()

    # build dictionary of labels
    for i in range(D):
        for f in feats[i]:
            feature_counts[f] += 1

        if labels[i] not in label_vocab:
            label_vocab[labels[i]] = lid
            lid += 1

    # build dictionary of features
    for f in feature_counts:
        if feature_counts[f] > MINIMUM_FEAT_COUNT and f not in feature_vocab:
            feature_vocab[f] = fmax
            fmax += 1

    # build reverse lookup for features and labels
    reverse_labels = [None]*lid
    for label in label_vocab:
        reverse_labels[label_vocab[label]] = label

    reverse_features = [None]*fmax
    for feature in feature_vocab:
        reverse_features[feature_vocab[feature]] = feature

    # X is a D-by-fmax matrix, where each row represents
    # features for a configuration / actions
    # Y is a list of labels for all configurations
    X = sparse.lil_matrix((D, fmax))
    Y = []
    for i in range(D):
        for f in feats[i]:
            if f in feature_vocab:
                fid = feature_vocab[f]
                X[i,fid] = 1
        Y.append(label_vocab[labels[i]])

    print ("Docs: ", D, "Features: ", fmax)
    log_reg = linear_model.LogisticRegression(C=L2_REGULARIZATION_STRENGTH,
                                              penalty='l2')

    clf = grid_search.GridSearchCV(log_reg, {'C':(1e0,1e1)}, n_jobs=10)
    log_reg = clf.fit(sparse.coo_matrix(X), Y).best_estimator_
    print ("Best C: %s" % clf.best_estimator_)
    print ("Accuracy: %s" % log_reg.score(sparse.coo_matrix(X), Y))

    return log_reg


def parse(toks, logreg):
    """
    parse sentence with trained model and return correctness measure
    """
    tokens, postags = {}, {}
    tokens[0] = "ROOT"
    postags[0] = "ROOT"

    heads, labels = {}, {}

    wbuffer, stack, arcs = [], [], []
    stack.append(0)

    for position in reversed(toks):
        # featurization
        (idd, tok, pos, head, lab) = position
        tokens[idd] = tok
        postags[idd] = pos

        # keep track of gold standards for performance evaluation
        heads[idd], labels[idd] = head, lab

        # update buffer
        wbuffer.append(idd)

    tree = {}
    while len(wbuffer) >= 0:
        if len(wbuffer) == 0 and len(stack) == 0: break
        if len(wbuffer) == 0 and len(stack) == 1 and stack[0] == 0: break

        feats = (featurize_configuration((wbuffer, stack, arcs), tokens, postags))
        X = sparse.lil_matrix((1, fmax))
        for f in feats:
            if f in feature_vocab:
                X[0,feature_vocab[f]]=1

        predictions = logreg.predict_proba(X)
        
        # your function will be called here
        action_to_tree(tree, predictions, wbuffer, stack, arcs)
        

    # correct_unlabeled: total number of correct (head, child) dependencies
    # correct_labeled: total number of correctly *labeled* dependencies
    correct_unlabeled, correct_labeled, total = 0, 0, 0

    for child in tree:
        (head, label) = tree[child]
        if head == heads[child]:
            correct_unlabeled += 1
            if label == labels[child]: correct_labeled += 1
        total += 1

    return [correct_unlabeled, correct_labeled, total]


def evaluate(filename, logreg):
    """
    Evaluate the performance of a parser against gold standard
    """
    with open(filename) as f:
        toks=[]
        totals = np.zeros(3)
        for line in f:
            cols=line.rstrip().split("\t")

            if len(cols) < 2: # end of a sentence
                if len(toks) > 0:
                    if is_projective(toks):
                        tots = np.array(parse(toks, logreg))
                        totals += tots
                        print ("%.3f\t%.3f\t%s" % (totals[0]/totals[2], totals[1]/totals[2], totals))
                    toks = []
                continue

            if cols[0].startswith("#"):
                continue

            idd, tok, pos, head, lab = int(cols[0]), cols[1], cols[4], int(cols[6]), cols[7]
            toks.append((idd, tok, pos, head, lab))


In [67]:
# ============================================================
# Run this cell to train the model
#
# NOTE: this takes a while to train, so you can change 
# the input to "train.projective.short.conll" to 
# train on only a subset of the data
#
# ============================================================

feats, labels = get_oracles("train.projective.conll")

In [70]:
logreg = train(feats, labels)

Docs:  371858 Features:  47679


/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


Best C: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Accuracy: 0.9213785907523839


In [77]:
evaluate("dev.projective.conll", logreg)

1.000	0.714	[7. 5. 7.]
0.769	0.692	[20. 18. 26.]
0.709	0.655	[39. 36. 55.]
0.714	0.661	[40. 37. 56.]
0.721	0.663	[62. 57. 86.]
0.683	0.625	[ 71.  65. 104.]
0.741	0.659	[100.  89. 135.]
0.755	0.662	[114. 100. 151.]
0.728	0.645	[123. 109. 169.]
0.708	0.624	[126. 111. 178.]
0.709	0.633	[141. 126. 199.]
0.704	0.637	[157. 142. 223.]
0.716	0.646	[174. 157. 243.]
0.713	0.647	[194. 176. 272.]
0.725	0.658	[206. 187. 284.]
0.719	0.647	[230. 207. 320.]
0.717	0.646	[251. 226. 350.]
0.724	0.653	[273. 246. 377.]
0.716	0.652	[300. 273. 419.]
0.715	0.650	[308. 280. 431.]
0.713	0.649	[321. 292. 450.]
0.715	0.650	[323. 294. 452.]
0.714	0.647	[344. 312. 482.]
0.705	0.644	[366. 334. 519.]
0.710	0.650	[380. 348. 535.]
0.710	0.652	[391. 359. 551.]
0.710	0.650	[406. 372. 572.]
0.705	0.645	[425. 389. 603.]
0.705	0.647	[437. 401. 620.]
0.707	0.649	[449. 412. 635.]
0.703	0.646	[453. 416. 644.]
0.695	0.639	[472. 434. 679.]
0.693	0.640	[494. 456. 713.]
0.692	0.641	[511. 473. 738.]
0.693	0.641	[534. 494. 771.]
0.6

0.713	0.662	[3449. 3200. 4836.]
0.713	0.662	[3460. 3211. 4852.]
0.714	0.663	[3477. 3228. 4870.]
0.714	0.663	[3486. 3237. 4881.]
0.714	0.663	[3490. 3241. 4889.]
0.713	0.663	[3497. 3248. 4902.]
0.714	0.663	[3505. 3254. 4910.]
0.714	0.663	[3514. 3263. 4919.]
0.715	0.664	[3522. 3270. 4927.]
0.715	0.664	[3529. 3277. 4934.]
0.715	0.664	[3531. 3279. 4936.]
0.716	0.665	[3534. 3282. 4939.]
0.716	0.665	[3552. 3299. 4959.]
0.715	0.665	[3574. 3321. 4996.]
0.716	0.666	[3592. 3339. 5016.]
0.716	0.666	[3614. 3359. 5047.]
0.717	0.666	[3636. 3379. 5073.]
0.717	0.666	[3639. 3382. 5076.]
0.717	0.666	[3641. 3384. 5078.]
0.716	0.666	[3654. 3397. 5103.]
0.716	0.666	[3660. 3403. 5112.]
0.715	0.665	[3662. 3405. 5119.]
0.716	0.665	[3668. 3409. 5125.]
0.715	0.665	[3681. 3421. 5148.]
0.715	0.665	[3689. 3427. 5156.]
0.716	0.665	[3698. 3436. 5165.]
0.716	0.665	[3700. 3438. 5167.]
0.716	0.665	[3701. 3439. 5168.]
0.715	0.664	[3706. 3443. 5185.]
0.715	0.664	[3708. 3445. 5187.]
0.715	0.664	[3711. 3448. 5190.]
0.715	0.

0.727	0.677	[5782. 5386. 7953.]
0.727	0.677	[5786. 5390. 7961.]
0.727	0.677	[5788. 5392. 7963.]
0.727	0.677	[5788. 5392. 7965.]
0.726	0.677	[5790. 5394. 7973.]
0.726	0.677	[5791. 5395. 7974.]
0.726	0.677	[5793. 5397. 7976.]
0.726	0.677	[5795. 5399. 7978.]
0.726	0.676	[5797. 5401. 7984.]
0.726	0.677	[5808. 5412. 7995.]
0.726	0.677	[5808. 5412. 7997.]
0.726	0.677	[5811. 5415. 8000.]
0.726	0.677	[5812. 5416. 8003.]
0.726	0.677	[5814. 5418. 8005.]
0.726	0.677	[5816. 5420. 8007.]
0.727	0.677	[5835. 5436. 8027.]
0.727	0.677	[5838. 5439. 8030.]
0.727	0.677	[5840. 5441. 8032.]
0.727	0.677	[5841. 5442. 8033.]
0.727	0.678	[5843. 5444. 8035.]
0.727	0.678	[5845. 5446. 8037.]
0.727	0.678	[5852. 5453. 8048.]
0.727	0.677	[5860. 5461. 8063.]
0.726	0.677	[5873. 5473. 8086.]
0.727	0.677	[5879. 5479. 8092.]
0.727	0.677	[5880. 5480. 8093.]
0.727	0.677	[5882. 5482. 8095.]
0.727	0.677	[5884. 5484. 8097.]
0.727	0.677	[5886. 5486. 8100.]
0.727	0.677	[5889. 5489. 8103.]
0.726	0.677	[5889. 5489. 8107.]
0.727	0.

0.733	0.683	[ 7742.  7210. 10563.]
0.732	0.682	[ 7752.  7220. 10583.]
0.733	0.682	[ 7765.  7233. 10600.]
0.732	0.682	[ 7776.  7243. 10616.]
0.733	0.682	[ 7777.  7244. 10617.]
0.733	0.683	[ 7787.  7254. 10628.]
0.732	0.682	[ 7787.  7254. 10633.]
0.732	0.682	[ 7793.  7260. 10642.]
0.732	0.682	[ 7806.  7272. 10659.]
0.732	0.682	[ 7830.  7296. 10694.]
0.732	0.682	[ 7846.  7311. 10714.]
0.732	0.683	[ 7883.  7347. 10763.]
0.732	0.683	[ 7885.  7349. 10766.]
0.733	0.683	[ 7895.  7358. 10776.]
0.733	0.683	[ 7898.  7361. 10779.]
0.733	0.683	[ 7907.  7370. 10788.]
0.733	0.683	[ 7924.  7386. 10811.]
0.733	0.683	[ 7928.  7390. 10815.]
0.733	0.683	[ 7929.  7391. 10821.]
0.733	0.683	[ 7930.  7392. 10822.]
0.733	0.683	[ 7953.  7413. 10851.]
0.733	0.683	[ 7958.  7418. 10859.]
0.733	0.683	[ 7965.  7425. 10866.]
0.733	0.683	[ 7980.  7440. 10889.]
0.733	0.683	[ 7991.  7451. 10905.]
0.733	0.683	[ 8001.  7461. 10919.]
0.733	0.683	[ 8005.  7465. 10926.]
0.733	0.683	[ 8006.  7466. 10927.]
0.733	0.683	[ 8007. 

0.726	0.677	[10439.  9733. 14375.]
0.726	0.677	[10445.  9739. 14381.]
0.726	0.677	[10451.  9745. 14392.]
0.726	0.677	[10459.  9753. 14404.]
0.726	0.677	[10479.  9771. 14427.]
0.726	0.677	[10487.  9779. 14440.]
0.726	0.677	[10496.  9788. 14458.]
0.726	0.677	[10517.  9807. 14481.]
0.726	0.677	[10520.  9810. 14484.]
0.726	0.677	[10522.  9812. 14487.]
0.726	0.677	[10528.  9818. 14493.]
0.726	0.677	[10541.  9830. 14512.]
0.726	0.677	[10545.  9834. 14516.]
0.726	0.677	[10563.  9850. 14540.]
0.727	0.678	[10570.  9857. 14548.]
0.727	0.678	[10580.  9867. 14561.]
0.727	0.678	[10600.  9886. 14587.]
0.727	0.678	[10614.  9899. 14602.]
0.727	0.678	[10617.  9902. 14605.]
0.727	0.678	[10619.  9904. 14608.]
0.727	0.678	[10624.  9909. 14613.]
0.727	0.678	[10642.  9926. 14634.]
0.727	0.678	[10645.  9929. 14640.]
0.727	0.678	[10647.  9931. 14642.]
0.727	0.678	[10655.  9939. 14650.]
0.727	0.678	[10661.  9944. 14657.]
0.727	0.678	[10662.  9945. 14658.]
0.727	0.678	[10663.  9946. 14659.]
0.728	0.679	[10671. 

0.729	0.680	[12776. 11907. 17521.]
0.729	0.680	[12781. 11912. 17526.]
0.729	0.680	[12788. 11919. 17536.]
0.729	0.680	[12804. 11933. 17556.]
0.729	0.680	[12811. 11939. 17566.]
0.729	0.680	[12825. 11952. 17583.]
0.730	0.680	[12851. 11975. 17615.]
0.730	0.680	[12862. 11984. 17627.]
0.730	0.680	[12866. 11988. 17631.]
0.730	0.680	[12869. 11991. 17634.]
0.730	0.680	[12889. 12008. 17654.]
0.730	0.680	[12903. 12022. 17676.]
0.730	0.680	[12915. 12034. 17690.]
0.730	0.680	[12929. 12048. 17711.]
0.730	0.680	[12942. 12058. 17724.]
0.730	0.680	[12946. 12061. 17734.]
0.730	0.680	[12973. 12085. 17767.]
0.730	0.680	[12986. 12097. 17781.]
0.730	0.680	[12986. 12097. 17783.]
0.730	0.680	[12991. 12102. 17788.]
0.730	0.680	[12997. 12108. 17794.]
0.730	0.680	[13008. 12118. 17808.]
0.730	0.680	[13015. 12125. 17821.]
0.731	0.680	[13041. 12147. 17851.]
0.731	0.680	[13048. 12154. 17861.]
0.731	0.681	[13067. 12173. 17883.]
0.731	0.681	[13082. 12188. 17906.]
0.731	0.681	[13093. 12198. 17917.]
0.731	0.681	[13099. 

0.735	0.684	[14603. 13605. 19877.]
0.735	0.684	[14607. 13609. 19884.]
0.735	0.685	[14613. 13615. 19890.]
0.734	0.684	[14625. 13627. 19913.]
0.734	0.684	[14628. 13630. 19916.]
0.734	0.684	[14634. 13636. 19930.]
0.734	0.684	[14644. 13646. 19941.]
0.734	0.684	[14647. 13649. 19944.]
0.735	0.684	[14658. 13659. 19956.]
0.735	0.684	[14660. 13661. 19958.]
0.735	0.685	[14669. 13669. 19967.]
0.735	0.685	[14672. 13672. 19970.]
0.734	0.684	[14684. 13683. 19997.]
0.734	0.684	[14693. 13692. 20006.]
0.735	0.685	[14705. 13703. 20018.]
0.735	0.685	[14713. 13711. 20026.]
0.735	0.685	[14715. 13713. 20028.]
0.735	0.685	[14716. 13714. 20031.]
0.735	0.685	[14722. 13720. 20037.]
0.735	0.685	[14730. 13726. 20050.]
0.735	0.685	[14735. 13729. 20055.]
0.735	0.685	[14739. 13733. 20059.]
0.735	0.684	[14739. 13733. 20064.]
0.735	0.685	[14749. 13743. 20075.]
0.735	0.685	[14759. 13753. 20085.]
0.735	0.685	[14766. 13759. 20093.]
0.735	0.685	[14771. 13764. 20098.]
0.735	0.685	[14781. 13774. 20110.]
0.735	0.685	[14787. 

0.736	0.687	[16408. 15309. 22291.]
0.736	0.687	[16411. 15312. 22294.]
0.736	0.687	[16428. 15327. 22315.]
0.736	0.687	[16444. 15343. 22339.]
0.736	0.687	[16447. 15346. 22342.]
0.736	0.687	[16461. 15360. 22359.]
0.736	0.687	[16475. 15374. 22373.]
0.736	0.687	[16481. 15380. 22380.]
0.737	0.687	[16490. 15388. 22389.]
0.737	0.687	[16493. 15391. 22392.]
0.737	0.687	[16508. 15406. 22412.]
0.737	0.687	[16527. 15424. 22439.]
0.737	0.687	[16532. 15428. 22444.]
0.737	0.687	[16538. 15434. 22450.]
0.737	0.688	[16547. 15441. 22459.]
0.737	0.688	[16561. 15455. 22477.]
0.737	0.688	[16570. 15464. 22491.]
0.737	0.688	[16574. 15468. 22495.]
0.737	0.688	[16585. 15478. 22507.]
0.737	0.688	[16601. 15492. 22532.]
0.737	0.687	[16605. 15496. 22542.]
0.737	0.687	[16607. 15498. 22544.]
0.737	0.687	[16623. 15514. 22566.]
0.737	0.687	[16628. 15518. 22575.]
0.736	0.687	[16632. 15521. 22583.]
0.737	0.687	[16641. 15529. 22593.]
0.737	0.687	[16644. 15532. 22596.]
0.737	0.688	[16656. 15543. 22608.]
0.737	0.688	[16663. 